In [ ]:
import pyarts
import numpy as np
import matplotlib.pyplot as plt

toa = 100e3
lat = 0
lon = 0
NQuad = 40

ws = pyarts.Workspace()

In [ ]:
# %% Sampled frequency range

ws.frequency_grid = [31.5e9, 165e9, 666e9]

# %% Species and line absorption

ws.absorption_speciesSet(species=["N2-SelfContStandardType", "O2-PWR98", "H2O-PWR98"])
ws.ReadCatalogData()
ws.propagation_matrix_agendaAuto()

## Setting ``propagation_matrix_scattering_spectral_agenda``

In [ ]:
ws.legendre_degree = 40
ws.propagation_matrix_scattering_spectral_agendaSet()

In [ ]:
ws.propagation_matrix_scatteringSpectralInit()

In [ ]:
ws.phase_matrix_scattering_spectral.shape

In [ ]:
#from pyarts import arts_agenda
#
#@arts_agenda
#def propagation_matrix_spectral(ws):
#    ws.propagation_matrix_scatteringInit(legendre_degree=ws.legendre_degree)
#    ws.propagation_matrix_scatteringAddSpectralScatteringSpeciesTRO()
#    ws.Ignore(ws.legendre_degree)

In [ ]:
from pyarts.arts import ParticleHabit
from pyarts.xml import load
scat_data_raw = load("testdata/scat_data.xml")
scat_data_meta = load("testdata/scat_meta.xml" )

t_grid = scat_data_raw[0][0].T_grid
f_grid = scat_data_raw[0][0].f_grid
rain_habit = ParticleHabit.from_legacy_tro(scat_data_raw[0], scat_data_meta[0]).to_tro_spectral(t_grid, f_grid, 40)

In [ ]:
from pyarts.arts import MGDSingleMoment, ScatteringSpeciesProperty, ParticulateProperty, AtmPoint
rain_first_moment = pyarts.arts.ScatteringSpeciesProperty("rain", pyarts.arts.ParticulateProperty("MassDensity"))
psd = MGDSingleMoment(rain_first_moment, "Wang16", 270, 300, False)

In [ ]:
from pyarts.arts import ScatteringHabit
rain = ScatteringHabit(rain_habit, psd)

In [ ]:
ws.scattering_species = [rain]

## Grids and Planet

In [ ]:
from pyarts.xml import load

p_grid = load("testdata/p_grid.xml")
t_field = load("testdata/t_field.xml")
z_field = load("testdata/z_field.xml")
vmr_field = load("testdata/vmr_field.xml")
pbf_field = load("testdata/particle_bulkprop_field.xml")
pbf_names = load("testdata/particle_bulkprop_names.xml")
ws.surface_fieldPlanet(option="Earth")
ws.surface_field[pyarts.arts.SurfaceKey("t")] = t_field[0, 0, 0]

In [ ]:
from pyarts.arts import GriddedField3, Tensor3, Vector

lat_grid = np.array([0.0])
lon_grid = np.array([0.0])
z_grid = z_field[..., 0, 0]

pressure = GriddedField3("p", p_grid[..., None, None], ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))
temperature = GriddedField3("t", t_field, ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))
n2 = GriddedField3("N2", vmr_field[0], ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))
o2 = GriddedField3("O2", vmr_field[1], ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))
h2o = GriddedField3("H2O", vmr_field[2], ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))
rwc = GriddedField3("RWC", pbf_field[0], ["alt", "lon", "lat"], (z_grid, lon_grid, lat_grid))

In [ ]:
ws.atmospheric_field["p"] = pressure
ws.atmospheric_field["t"] = temperature
ws.atmospheric_field["N2"] = n2
ws.atmospheric_field["O2"] = o2
ws.atmospheric_field["H2O"] = h2o
ws.atmospheric_field[rain_first_moment] = rwc
ws.atmospheric_field.top_of_atmosphere = 12.0e3

## Checks and settings

In [ ]:
ws.spectral_radiance_unit = "Tb"
ws.spectral_radiance_space_agendaSet(option="UniformCosmicBackground")
ws.spectral_radiance_surface_agendaSet(option="Blackbody")

#  Core Disort calculations

In [ ]:
ws.spectral_radiance_unit = "Tb"
ws.spectral_radiance_space_agendaSet(option="UniformCosmicBackground")
ws.spectral_radiance_surface_agendaSet(option="Blackbody")

# %% Core Disort calculations

def calculate_tbs_disort():
    ws.disort_settings_agendaSetup(scattering_setting="ScatteringSpecies")
    ws.disort_spectral_radiance_fieldProfile(
        longitude=lon,
        latitude=lat,
        disort_quadrature_dimension=NQuad,
        disort_legendre_polynomial_dimension=1,
        disort_fourier_mode_dimension=1,
    )
    disort_stokes = [[ws.disort_spectral_radiance_field[f_ind, 0, 0, 19], 0.0, 0.0, 0.0] for f_ind in range(3)]
    ws.spectral_radiance = disort_stokes
    ws.spectral_radianceApplyUnit()
    return ws.spectral_radiance.value
    

In [ ]:
calculate_tbs_disort()